In [ ]:
import re
import pandas as pd

sql_list = ['10b', '10c', '11b', '11c', '11d', '12b', '12c', '13b',
            '13c', '13d', '14b', '14c', '15b', '15c', '15d', '16b',
            '16c', '16d', '17b', '17c', '17d', '17e', '17f', '18b',
            '18c', '19b', '19c', '19d', '1b', '1c', '1d', '20b', '20c',
            '21b', '21c', '22b', '22c', '22d', '23b', '23c', '24b',
            '25b', '25c', '26b', '26c', '27b', '27c', '28b', '28c',
            '29b', '29c', '2b', '2c', '2d', '30b', '30c', '31b', '31c',
            '32b', '33b', '33c', '3b', '3c', '4b', '4c', '5b', '5c',
            '6b', '6c', '6d', '6e', '6f', '7b', '7c', '8b', '8c',
            '8d', '9b', '9c', '9d']

execution_pattern = re.compile(r"(\w+)\s+Execution time:\s+([\d.]+)\s+\(predicted\s+([\d.]+)\)")
expert_pattern = re.compile(r"Expert plan:\s+latency,\s+predicted,\s+hint\n\s+([\d.]+)\s+\(predicted\s+([\d.]+)\)")

In [ ]:
def process_log(file_path):
    data = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    for i, line in enumerate(lines):
        exec_match = execution_pattern.search(line)
        if exec_match and exec_match.group(1) in sql_list:
            sql_name = exec_match.group(1)
            execution_time = float(exec_match.group(2))
            predicted_execution = float(exec_match.group(3))
            
            if i + 1 < len(lines):
                expert_match = expert_pattern.search(lines[i + 1] + (lines[i + 2] if i + 2 < len(lines) else ''))
                if expert_match:
                    expert_latency = float(expert_match.group(1))
                    data.append((sql_name, execution_time, predicted_execution, expert_latency))

    return pd.DataFrame(data, columns=['SQL_Name', 'Execution_Time', 'Predicted_Execution', 'Expert_Latency'])


In [ ]:
def process(filename):
    filename_a = filename + "-True.log"
    filename_b = filename + "-False.log"

    df_a = process_log(filename_a)
    df_b = process_log(filename_b)
    merged_df = pd.concat([df_a.set_index('SQL_Name'), df_b.set_index('SQL_Name')], axis=1, keys=['True', 'False'])

    success_df = merged_df[merged_df[('True', 'Execution_Time')] < merged_df[('False', 'Execution_Time')]]

    success_df.reset_index(inplace=True)
    success_df

    valid_success_df = success_df[success_df[('True', 'Predicted_Execution')] != success_df[('False', 'Predicted_Execution')]]

    valid_success_df.reset_index(inplace=True)
    valid_success_df

    valid_success_df.columns = valid_success_df.columns.set_levels(
        ['True - with cp assist' if level == 'True' else 'False - without cp assist' if level == 'False' else level for level in valid_success_df.columns.levels[0]], 
        level=0
    )
    pd.set_option('display.expand_frame_repr', False)
    display(valid_success_df)

In [ ]:
process("balsa99")

In [ ]:
process("balsa149")